In [ ]:
from pathlib import Path as path
import spectrochempy as scp
from spectrochempy import ur
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import matplotlib.gridspec as gridspec
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.colorbar import Colorbar
import sys

In [ ]:
# 画图的初始设置
plt.style.use(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-python\Figure\liuchzzyy.mplstyle')
# display(plt.style.available)

# 颜色设定
sys.path.append(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Python\Figure')
from colors import tol_cmap, tol_cset
colors = list(tol_cset('vibrant'))
if r'sunset' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('sunset'))
if r'rainbow_PuRd' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('rainbow_PuRd')) # 备用 plasma

# 设置数学字体 Set math font
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Arial'
mpl.rcParams['mathtext.it'] = 'Arial:italic'
mpl.rcParams['mathtext.bf'] = 'Arial:bold'
mpl.rcParams['mathtext.sf'] = 'Arial'
mpl.rcParams['mathtext.tt'] = 'Arial'
mpl.rcParams['mathtext.cal'] = 'Arial'
mpl.rcParams['mathtext.default'] = 'regular'

# 输出的文件夹
path_out = path(r"C:\Users\chengliu\Desktop\Figure")

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XAS\ExSitu\αMnO2\XAS\EMD-SK')
df = pd.read_csv(path.joinpath(path_file, r'EMD_Electrode.chir2_mag'), comment='#', sep='\s+', index_col=None, header=None)
df

In [ ]:
# df.iloc[[78,98], 3:].T.to_csv(path.joinpath(path_out, r'EXAFS_intensity.csv'), index=False, header=None)

In [ ]:
Mn_NormalData = xr.Dataset(
    data_vars={
        'Spectrum': (['r', 'spectrum_number'], df.iloc[:, 1:]),
        'Reference': (['r', 'reference_number'], df.iloc[:, 1:3]),
        },
    coords={
        'spectrum_number': df.columns[1:],
        'r': df.iloc[:,0],
        'reference_number': [r'$\alpha$-MnO$_2$',r'0.2M Mn2+'],
        },)

data = scp.NDDataset(data=Mn_NormalData['Spectrum'].data.T,
                     name="opXAS_Mn_NOTOs_2024",
                     author="Cheng Liu, Ashely, and Dino Tonti",
                     description="XopXAS_Mn_NOTOs_2024",
                     history="creation",
                     title='absorption',
                     units=ur.absorbance)

data.y = scp.Coord.arange(Mn_NormalData['Spectrum'].shape[1], title='sperctum number')
data.y.labels = Mn_NormalData.coords['spectrum_number']
data.x = scp.Coord(Mn_NormalData.coords['r'], title='r', units=None)

ref = scp.NDDataset(data=Mn_NormalData['Reference'].data.T, name="ref_NOTOs_2024", units=ur.absorbance)
ref.y = scp.Coord.arange(Mn_NormalData['Reference'].shape[1], title='sperctum number')
ref.x = scp.Coord(Mn_NormalData.coords['r'], title='r', units=None)

data = data[:11].copy(True)
_ = data.plot(x='r', y='spectrum_number', xlim=(0.0, 6.0), clear=True)
_ = ref.plot(x='r', y='spectrum_number', ls='--', xlim=(0.0, 6.0), clear=False)

In [ ]:
peakslist = [s.find_peaks(distance=0.5, height=0.0, width=0.1, rel_height=0.8, wlen=2)[0] for s in data]
properties = [s.find_peaks(distance=0.5, height=0.0, width=0.1, rel_height=0.8, wlen=2)[1] for s in data]

isp = []
for file in properties:
    left_base = file['left_bases']
    right_base = file['right_bases']
    merged_bases = sorted(list(set(left_base + right_base)))
    targets = [2.0, 2.8, 3.4]
    closest_values = {target: min(merged_bases, key=lambda x: abs(x - target)) for target in targets}
    isp.append(closest_values)

isp = pd.DataFrame(isp)
isp
from scipy.integrate import simpson
area = []
for i, file in enumerate(data):
    file1 = file[:,isp.iloc[i, 0]:isp.iloc[i, 1]]
    file2 = file[:,isp.iloc[i, 1]:isp.iloc[i, 2]]
    intsimps = simpson(file1, axis=1)[0]
    intsimps2 = simpson(file2, axis=1)[0]
    area.append([intsimps, intsimps2])
area = pd.DataFrame(area, columns=['Edge_area', 'Corner_area'])
area['ratio'] = area.iloc[:, 0] / (area.iloc[:, 1] + area.iloc[:, 0])
area = area.apply(np.round, decimals=3)
area['Samples'] = data.y.labels.astype(np.int8)
area.to_csv(path.joinpath(path_out, r'EXAFS_FD_Rbkg_1.3_area.csv'), index=False, header=True)
area

In [ ]:
_ = ref[0].plot()
plt.show()

In [ ]:
peakslist = [s.find_peaks(distance=0.5, height=0.3, width=0.1, rel_height=0.8, wlen=2)[0] for s in ref]
properties = [s.find_peaks(distance=0.5, height=0.3, width=0.1, rel_height=0.8, wlen=2)[1] for s in ref]

isp = []
for file in properties:
    left_base = file['left_bases']
    right_base = file['right_bases']
    merged_bases = sorted(list(set(left_base + right_base)))
    targets = [2.0, 2.8, 3.4]
    closest_values = {target: min(merged_bases, key=lambda x: abs(x - target)) for target in targets}
    isp.append(closest_values)

isp = pd.DataFrame(isp)
isp

from scipy.integrate import simpson
area = []
for i, file in enumerate(ref[1:]):
    file1 = file[:,isp.iloc[i, 0]:isp.iloc[i, 1]]
    file2 = file[:,isp.iloc[i, 1]:isp.iloc[i, 2]]
    intsimps = simpson(file1, axis=1)[0]
    intsimps2 = simpson(file2, axis=1)[0]
    area.append([intsimps, intsimps2])
area = pd.DataFrame(area, columns=['Edge_area', 'Corner_area'])
area['ratio'] = area.iloc[:, 0] / (area.iloc[0, 1] + area.iloc[0, 0])
area = area.apply(np.round, decimals=3)
area